In [1]:

# returnes matrix mu and list of squared euclidean norms
#   of the vectors in Gram-Schmidt orthogonalized basis
#   (Cholesky decomposition avoiding use of irrationals)
# G ... Gram matrix to be decomposed
def chol_decomp(G, cols=-1):
    rows = G.nrows()
    if cols < 0:
        cols = rows
    mu = matrix(QQ, rows, cols)
    squares = vector(QQ, cols)
    for y in range(rows):
        over_cols = (y>=cols)
        for x in range(cols if over_cols else y):
            t = 0
            for i in range(x):
                t += mu[y,i]*mu[x,i]*squares[i]
            mu[y,x] = (G[y,x]-t)/squares[x]
        if not over_cols:
            t = 0
            for i in range(y):
                t += mu[y,i]*mu[y,i]*squares[i]
            mu[y,y] = 1
            squares[y] = G[y,y]-t
    return (mu, squares)

# auxiliary function: swaps i-th row with the previous one
def swap_rows_in_mu(mu, B, i, n):
    mu[i-1], mu[i] = mu[i], mu[i-1]
    mu[i,i], mu[i-1,i] = 1, 0
    r = mu[i-1,i-1]
    mu[i-1,i-1] = 1
    temp = B[i-1]*r**2 + B[i]
    mu[i,i-1] = r*B[i-1]/temp
    B[i] = B[i-1]*B[i]/temp
    B[i-1] = temp
    for j in range(i+1, n):
        mu[j,i-1], mu[j,i] = mu[i,i-1]*mu[j,i-1]+mu[j,i]*(1-r*mu[i,i-1]), \
                           mu[j,i-1]-r*mu[j,i]

# auxiliary function: the inner cycle of LLL
def LLL_reduce_row(b, mu, i, h):
    for j in range(h-1, -1, -1):
        t = round(mu[i, j])
        for k in range(j+1):
            mu[i, k] -= t*mu[j, k]
        b[i] -= t*b[j]

# auxiliary function: LLL reduction algorithm
# b ... integer basis (does'n have to be square matrix)
# mu ... Gram-Schmidt mu matrix (doesn't have to be square matrix)
# B ... squared norms of Gram-Schmidt orthogonalized vectors
# n ... number of rows of b to be reduced (usually b.nrows())
# h ... index of first row not to be swapped (only reduced)
def LLL_aux(b, delta, mu, B, n, h):
    i = 1
    flag = True
    while i < h:
        if flag:
            LLL_reduce_row(b, mu, i, i)
        if i > 0 and (delta-mu[i,i-1]**2)*B[i-1] > B[i]:
            b[i], b[i-1] = b[i-1], b[i]
            swap_rows_in_mu(mu, B, i, n)
            i -= 1
            flag = False
        else:
            i += 1
            flag = True
    while i < n:  # performed only if n>h
        LLL_reduce_row(b, mu, i, h)
        i += 1
    return b, mu, B

# classical LLL algorithm
# last row of b can be used as a target point in Babai's algorithm
def LLL_own(b, delta=3/4, babai_bool=False):
    mu, B = chol_decomp(b*b.transpose(), b.nrows()-babai_bool)
    return LLL_aux(copy(b), delta, mu, B, b.nrows(), b.nrows()-babai_bool)[0]

# LLL for triangular input basis
def LLL_triangular_input(b, delta=3/4, babai_bool=False):
    B = vector(QQ, [b[j,j]**2 for j in range(b.ncols())])
    mu = matrix(QQ, b)
    for j in range(mu.ncols()):
        r = mu[j,j]
        for i in range(j, mu.nrows()):
            mu[i,j] /= r
    return LLL_aux(copy(b), delta, mu, B, b.nrows(), b.nrows()-babai_bool)[0]


In [3]:
import random  # FIXME put import as, abych nebyla prase
# define seed to fix the random matrix
random.seed(1)

# dimension, perimeter for input values
n = 10
perimeter = 50 

list = [randint(-perimeter, perimeter) for _ in range(n*n)]

# basis matrix of a lattice
B = matrix(ZZ, n, n, list)
B

[ 40  -2   6 -25 -48 -17 -23  47  33  29]
[  5  29 -44   3  -7  16  29  -3  15  11]
[  6  16 -27  25 -11  13  17 -44  29  24]
[ 10  34 -25  18 -30  20  -2 -24 -22  -2]
[ 39  30  37  27  14 -50  33 -10  15  46]
[ 32 -31  24 -28  26  38 -21  50  19   5]
[-27  -8  22 -30  39 -13 -34   5  14 -14]
[ -5 -18  -8 -49 -39 -12  34 -28   5   7]
[-50   3 -37 -41 -29   9 -21 -23  28   6]
[ 35  -8  -2  24 -10  19 -43 -42   8  31]

In [15]:
B_own = LLL_own(B)
norms = []
for i in range(n):
    # print(B_own[i], end="  ")
    norms.append(B_own[i].norm())
print(norms)
    
B_own

[sqrt(3158), 14*sqrt(22), sqrt(4121), 5*sqrt(129), 2*sqrt(853), sqrt(2863), 2*sqrt(913), 22*sqrt(7), sqrt(5262), 3*sqrt(515)]


[  1 -13  17  22  -4  -3 -12 -41  14  13]
[  5  29 -44   3  -7  16  29  -3  15  11]
[  5   5  19  15 -23   4 -31 -21 -37 -13]
[-17  26  -3 -12   9   7 -36 -19  -8 -16]
[  2  -4  23 -12  31  14  31   9 -14  18]
[ -8   2 -18  12  -2 -36  -8 -27  -3  15]
[ 21   4 -16 -23  -9 -32  19 -12  20 -20]
[ 34   5 -19   2  -6  22 -31  -1  -6  18]
[  1  49  20 -33 -32   2   1  14  -5  11]
[-26 -22   8 -26 -30  20  15 -16 -15  27]

In [14]:
B_local = B.LLL()
norms_2 = []
for i in range(n):
    # print(B_local[i], end="  ")
    norms_2.append(B_local[i].norm())
print(norms_2)

B_local
    

[sqrt(2863), sqrt(3158), 2*sqrt(853), 5*sqrt(129), sqrt(4121), 2*sqrt(913), 22*sqrt(7), 14*sqrt(22), sqrt(5262), 3*sqrt(515)]


[ -8   2 -18  12  -2 -36  -8 -27  -3  15]
[  1 -13  17  22  -4  -3 -12 -41  14  13]
[  2  -4  23 -12  31  14  31   9 -14  18]
[-17  26  -3 -12   9   7 -36 -19  -8 -16]
[  5   5  19  15 -23   4 -31 -21 -37 -13]
[ 21   4 -16 -23  -9 -32  19 -12  20 -20]
[ 34   5 -19   2  -6  22 -31  -1  -6  18]
[  5  29 -44   3  -7  16  29  -3  15  11]
[  1  49  20 -33 -32   2   1  14  -5  11]
[-26 -22   8 -26 -30  20  15 -16 -15  27]

50 x 50 dense matrix over Real Field with 53 bits of precision (use the '.str()' method to see the entries)